In [1]:
import os
os.getcwd()

'/data'

In [2]:
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Activation, Dropout, Flatten, Dense
from keras import backend as K

from keras import optimizers

from keras.models import load_model

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
#from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model 
#from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping


In [4]:
# create the base pre-trained model
#base_model = InceptionV3(weights='imagenet', include_top=False)

In [5]:
#import tensorflow as tf
#sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 3541814423885478289
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11022306509
locality {
  bus_id: 1
  links {
  }
}
incarnation: 5726818889170437136
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:1e.0, compute capability: 3.7"
]


In [6]:
img_width, img_height = 299, 299
train_data_dir = "/data/fgcvx/train/images/"
validation_data_dir = "/data/fgcvx/train/images/"
nb_train_samples = 81417
nb_validation_samples = 8343 

In [7]:
nr_categories = 1394
batch_size = 16
epochs = 50

In [8]:
# model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.0001, momentum=0.9), metrics=["accuracy"])
#base_model.output

In [9]:
# add a global spatial average pooling layer
#x = base_model.output
#x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
#x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
#predictions = Dense(nr_categories, activation='softmax')(x)



In [10]:
# this is the model we will train
#model = Model(inputs=base_model.input, outputs=predictions)
model = load_model("/data/inception_fungi_all.h5")

In [11]:
for i, layer in enumerate(model.layers):
   print(i, layer.name)

0 input_1
1 conv2d_1
2 batch_normalization_1
3 activation_1
4 conv2d_2
5 batch_normalization_2
6 activation_2
7 conv2d_3
8 batch_normalization_3
9 activation_3
10 max_pooling2d_1
11 conv2d_4
12 batch_normalization_4
13 activation_4
14 conv2d_5
15 batch_normalization_5
16 activation_5
17 max_pooling2d_2
18 conv2d_9
19 batch_normalization_9
20 activation_9
21 conv2d_7
22 conv2d_10
23 batch_normalization_7
24 batch_normalization_10
25 activation_7
26 activation_10
27 average_pooling2d_1
28 conv2d_6
29 conv2d_8
30 conv2d_11
31 conv2d_12
32 batch_normalization_6
33 batch_normalization_8
34 batch_normalization_11
35 batch_normalization_12
36 activation_6
37 activation_8
38 activation_11
39 activation_12
40 mixed0
41 conv2d_16
42 batch_normalization_16
43 activation_16
44 conv2d_14
45 conv2d_17
46 batch_normalization_14
47 batch_normalization_17
48 activation_14
49 activation_17
50 average_pooling2d_2
51 conv2d_13
52 conv2d_15
53 conv2d_18
54 conv2d_19
55 batch_normalization_13
56 batch_norma

In [13]:
# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in model.layers:
    layer.trainable = False


In [14]:
# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=["accuracy"])


In [15]:
train__val_datagen = ImageDataGenerator(rescale = 1./255, 
                                        shear_range=0.2,
                                        zoom_range=0.2,
                                        horizontal_flip=True,
                                        validation_split=0.1)

In [16]:
img_width, img_height = 128, 128
train_generator = train__val_datagen.flow_from_directory(
        train_data_dir,  # this is the target directory
        target_size=(img_width, img_height),  # all images will be resized to 250x250
        batch_size=batch_size,
        #save_to_dir = "/data/fg/train",
        subset="training",
        shuffle=True, seed=13,
        class_mode='categorical')

Found 81417 images belonging to 1394 classes.


In [17]:
validation_generator = train__val_datagen.flow_from_directory(
        train_data_dir,  # this is the target directory
        target_size=(img_width, img_height),  # all images will be resized to 250x250
        subset="validation",
        #save_to_dir = "/data/fg/val",
        batch_size=batch_size,
        shuffle=True, seed=13,
        class_mode='categorical')

Found 8343 images belonging to 1394 classes.


In [18]:
# Save the model according to the conditions  
checkpoint = ModelCheckpoint("inception_fungi_features.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=5, verbose=1, mode='auto')


In [19]:
from time import time
tensorboard = TensorBoard(log_dir="logs/{}".format(time()), histogram_freq=0, write_graph=True, write_images=True)

In [ ]:
#import numpy as np
#validation_generator.class_indices

In [ ]:
len(os.listdir("/data/fgcvx/train/images/10000_Abortiporus_biennis"))

In [ ]:
import numpy as np

In [ ]:
train_labels = []
species = os.listdir("/data/fgcvx/train/images/")

for specie in species:
    n = len(os.listdir("/data/fgcvx/train/images/{}".format(specie)))
    train_labels.append([train_generator.class_indices[specie]]*n)
    
    

In [ ]:
val_labels = []
species = os.listdir("/data/fgcvx/train/images/")

for specie in species:
    n = len(os.listdir("/data/fgcvx/train/images/{}".format(specie)))
    val_labels.append([validation_generator.class_indices[specie]]*n)
    
    

In [ ]:
import itertools
val_labels_ = list(itertools.chain(*val_labels))
train_labels_ = list(itertools.chain(*train_labels))

In [ ]:
len(train_labels_)

In [ ]:
tr = np.array(train_labels_)

In [ ]:
vl = np.array(val_labels_)


In [ ]:
tr == vl

In [ ]:
bottleneck_features_train = model.predict_generator(train_generator)

In [ ]:
bottleneck_features_val = model.predict_generator(validation_generator)

In [ ]:
with open('/data/bottleneck_features_train_fg_inceptionv3.npy', 'wb') as fh:
    np.save(fh, str(bottleneck_features_train))
bottleneck_features_train.shape

In [ ]:
with open('/data/bottleneck_features_val_fg_inceptionv3.npy', 'wb') as fh:
    np.save(fh, str(bottleneck_features_val))
bottleneck_features_val.shape

In [ ]:
print(1)

In [ ]:
validation_data = bottleneck_features_val
train_data = bottleneck_features_train
#validation_labels = np.array([0] * 400 + [1] * 400)

In [ ]:
tr

In [ ]:
#train_data = np.load(open('/data/bottleneck_features_train_fg_inceptionv3.npy', "rb"))
# the features were saved in order, so recreating the labels is easy
#train_labels = np.array([0] * 1000 + [1] * 1000)
train_labels = tr
validation_labels = vl


In [ ]:
print(len(train_labels))
print(len(val_labels))

In [ ]:
train_data.shape[1:]
bottleneck_features_train.shape

In [ ]:
x = model.output
# x = Flatten()(x)

In [ ]:
x = Flatten()(x)
x.shape

In [ ]:
#train_data = np.load(open('/data/bottleneck_features_train_fg_inceptionv3.npy', "rb"))
# the features were saved in order, so recreating the labels is easy
#train_labels = np.array([0] * 1000 + [1] * 1000)

model = Sequential()
model.add(Flatten(input_shape=train_data.shape[1:]))
model.add(Dense(nr_categories*2, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(nr_categories, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(train_data, train_labels,
          epochs=50,
          batch_size=batch_size,
          validation_data=(validation_data, validation_labels))
model.save_weights('bottleneck_fc_model_fg.h5')



In [20]:
model.fit_generator(
train_generator,
#samples_per_epoch = nb_train_samples,
steps_per_epoch = nb_train_samples/batch_size,    
epochs = epochs,
validation_data = validation_generator,
#nb_val_samples = nb_validation_samples,
validation_steps = nb_validation_samples/batch_size,    
callbacks = [checkpoint, early, tensorboard])

Epoch 1/50
3049/5088 [================>.............] - ETA: 14:09 - loss: 7.0151 - acc: 0.0078

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:747: UserWarning: Possibly corrupt EXIF data.  Expecting to read 4718592 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))


5088/5088 [============================>.] - ETA: 0s - loss: 7.0142 - acc: 0.0075

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:764: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


5089/5088 [==============================] - 2342s 460ms/step - loss: 7.0143 - acc: 0.0075 - val_loss: 6.9805 - val_acc: 0.0066

Epoch 00001: val_acc improved from -inf to 0.00659, saving model to inception_fungi_features.h5
Epoch 2/50
5089/5088 [==============================] - 2336s 459ms/step - loss: 7.0143 - acc: 0.0074 - val_loss: 6.9871 - val_acc: 0.0066

Epoch 00002: val_acc did not improve from 0.00659
Epoch 3/50
5089/5088 [==============================] - 2334s 459ms/step - loss: 7.0140 - acc: 0.0072 - val_loss: 6.9948 - val_acc: 0.0062

Epoch 00003: val_acc did not improve from 0.00659
Epoch 4/50
5089/5088 [==============================] - 2334s 459ms/step - loss: 7.0156 - acc: 0.0074 - val_loss: 6.9757 - val_acc: 0.0064

Epoch 00004: val_acc did not improve from 0.00659
Epoch 5/50
5089/5088 [==============================] - 2336s 459ms/step - loss: 7.0156 - acc: 0.0072 - val_loss: 6.9777 - val_acc: 0.0064

Epoch 00005: val_acc did not improve from 0.00659
Epoch 6/50
5089

In [21]:
# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the remaining top layers.

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers[:249]:
   layer.trainable = False
for layer in model.layers[249:]:
   layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=["accuracy"])

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
model.fit_generator(
train_generator,
#samples_per_epoch = nb_train_samples,
steps_per_epoch = nb_train_samples/batch_size,    
epochs = epochs,
validation_data = validation_generator,
#nb_val_samples = nb_validation_samples,
validation_steps = nb_validation_samples/batch_size,    
callbacks = [checkpoint, early, tensorboard])

NameError: name 'base_model' is not defined